In [1]:
import os
import sys
import time
import datetime

import random
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Flatten, BatchNormalization, \
Conv2D, MaxPool2D, Dropout, Input, LeakyReLU, AveragePooling2D

from keras.optimizers import Adam, SGD , RMSprop
from keras.losses import mae, sparse_categorical_crossentropy, binary_crossentropy

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet import preprocess_input

from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping

from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.resnet import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.densenet import DenseNet201
from keras.applications.nasnet import NASNetMobile, NASNetLarge

import warnings
warnings.filterwarnings('ignore')

In [2]:
def timestamp(x):
    
    if x==1:
        return 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x==2:    
        return 'Timestamp: {:%Y-%b-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x==3:  
        return 'Date now: %s' % datetime.datetime.now()
    if x==4:  
        return 'Date today: %s' % datetime.date.today()  

In [3]:
dataset_path = os.path.join(os.getcwd(), 'Dataset')

train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')

#print(train_dir)

In [4]:
def plot_img(file, dirs = train_dir):
  
    img_path = os.path.join(dirs, file)
    img = plt.imread(img_path)
    #img = img.numpy().transpose((1, 2, 0))
    plt.imshow(img, aspect = None)

    plt.xticks([])
    plt.yticks([])

def img_path(dir = train_dir, count = 5):

    files = random.sample(os.listdir(dir), count)
    ncol, nrow = 5, 3
  
    FIGSIZE = (20, 15)
    plt.figure(figsize=FIGSIZE)

    for i, file in enumerate(files):
        plt.subplot(nrow, ncol, i+1)
        path = os.path.join(dir, file)
        plot_img(file, dir)
    plt.tight_layout()
    plt.show()

def label_path(dir = train_dir, count = 5):
    labels = os.listdir(dir)
    for label in labels:
        print(label)
        label_path = os.path.join(dir, label)
        img_path(dir = label_path, count = 5)

In [5]:
#label_path(train_dir, 5)

In [6]:
def create_data(train_path = None, test_path = None, batch_size = 64, target_size = (300, 300), class_mode = 'categorical'):

    print('Creating and preprocessing data batches for training and testing!')
    print('-' * 65)

    TRAIN_BATCH_SIZE, TEST_BATCH_SIZE = batch_size, batch_size
    RESCALE = 1.0/255

    train_shuffle = True
    val_shuffle = True
    test_shuffle = False

    train_generator = ImageDataGenerator(
        horizontal_flip=True,
        rotation_range=30,
        rescale=RESCALE,
        validation_split=0.25
    )

    test_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

    train_data = train_generator.flow_from_directory(
        train_path,
        target_size = target_size,
        class_mode = class_mode,
        batch_size = TRAIN_BATCH_SIZE,
        shuffle = train_shuffle,
        seed = 42,
        subset = 'training'
    )

    validation_data = train_generator.flow_from_directory(
        train_path,
        target_size = target_size,
        class_mode = class_mode,
        batch_size = TRAIN_BATCH_SIZE,
        shuffle = val_shuffle,
        seed = 42,
        subset = 'validation'
    )

    test_data = test_generator.flow_from_directory(
        test_path,
        target_size = target_size,
        batch_size = TEST_BATCH_SIZE,
        shuffle = test_shuffle
    )

    train_step = train_data.n//train_data.batch_size
    val_step = validation_data.n//validation_data.batch_size

    return train_data, validation_data, test_data, train_step, val_step

In [7]:
def create_model(model_name, input_shape = (256, 256, 3), num_class = 3):
    input = Input(input_shape)

    if model_name == 'ResNet50':
        base_model = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)
    elif model_name == 'VGG16':
        base_model = VGG16(include_top = False, weights = 'imagenet', input_shape = input_shape)
    elif model_name == 'NASNetMobile':
        base_model = NASNetMobile(include_top = False, weights = 'imagenet', input_shape = input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    head_model = base_model.output
  # head_model = AveragePooling2D(pool_size=(4, 4))(head_model)
    head_model = Flatten(name="flatten")(head_model)
    head_model = Dense(64, activation="relu")(head_model)
    head_model = LeakyReLU(alpha=0.4)(head_model)
    head_model = Dropout(0.2)(head_model)
    
    if num_class > 1:
        head_model = Dense(num_class, activation="softmax")(head_model)
    else:
        head_model = Dense(1, activation="sigmoid")(head_model)

    model = Model(base_model.input, head_model)

    model.summary()

    return model

In [8]:
input_shape = (224, 224, 3)
num_class = 3
model_name = 'VGG16'

print('Loading Base Model')

vggnet = create_model(model_name = model_name, input_shape = input_shape, num_class = num_class)

print('Model Loaded at ', timestamp(1))

Loading Base Model
Metal device set to: Apple M1


2022-12-24 19:29:20.788955: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-24 19:29:20.789007: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
batch_size = 64
target_size = (224, 224)

train_data, val_data, test_data, train_step, val_step = create_data(train_path = train_dir, test_path = test_dir, batch_size = batch_size, 
                                                                    target_size=target_size, class_mode='categorical')

Creating and preprocessing data batches for training and testing!
-----------------------------------------------------------------
Found 9093 images belonging to 3 classes.
Found 3030 images belonging to 3 classes.
Found 3030 images belonging to 3 classes.


In [10]:
EPOCHS = 10
ILR = 0.001
VERBOSE = 1

optimizer = Adam(learning_rate = ILR)
loss = 'categorical_crossentropy'
metrics = ['accuracy']

print('Compiling Model')
print('-' * 15)

vggnet.compile(optimizer = optimizer, loss = loss, metrics = metrics)

Compiling Model
---------------


In [11]:
loss0, accuracy0 = vggnet.evaluate(val_data)

print(f'Initial Loss: {loss0:.2f} Initial Accuracy: {accuracy0:.2f}')

2022-12-24 19:29:44.633335: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-24 19:29:44.799431: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


48/48 [==============================] - 105s 2s/step - loss: 0.9664 - accuracy: 0.6670
Initial Loss: 0.97 Initial Accuracy: 0.67


In [12]:
dir_name = 'Model Checkpoint'
file_path = os.path.join(os.getcwd(), dir_name)

if not os.path.exists(file_path):
    print('Directory does not exist. Making it now!')
    os.mkdir(file_path)

In [13]:
def scheduler(epoch, lr = ILR):
    
    if epoch % 3 == 0:
        return lr * 0.9
    else:
        return lr
    
def set_callbacks(verbose = 0):
    
    model_checkpoint = ModelCheckpoint(filepath = file_path,
                                       monitor = 'val_accuracy',
                                       verbose = 0,
                                       save_best_only = True)
    
    early_stopping = EarlyStopping(monitor = 'val_loss',
                                   patience = 2,
                                   verbose = verbose,
                                   restore_best_weights = True)
    
    learn_scheduler = LearningRateScheduler(scheduler,
                                            verbose = verbose)
    
    callbacks = [model_checkpoint, learn_scheduler, early_stopping]
    
    print('Setting Callbacks...')
    
    return callbacks

In [15]:
callbacks = set_callbacks(verbose = 1)

print("Starting Training ...\n")

start_time = time.time()

history = vggnet.fit(
    train_data,
    steps_per_epoch=train_step,
    epochs = EPOCHS,
    verbose = VERBOSE,
    callbacks = callbacks,
    validation_data = val_data,
    validation_steps = val_step
)

elapsed_time = time.time() - start_time
elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

print("\nElapsed Time: " + elapsed_time)
print("Completed Model Trainning", timestamp(1))

Setting Callbacks...
Starting Training ...


Epoch 1: LearningRateScheduler setting learning rate to 0.0009000000427477062.
Epoch 1/10
142/142 [==============================] - ETA: 0s - loss: 0.4776 - accuracy: 0.7829

INFO:tensorflow:Assets written to: /Users/anuragkotiyal/Desktop/Projects/COVID Detection/Model Checkpoint/assets


INFO:tensorflow:Assets written to: /Users/anuragkotiyal/Desktop/Projects/COVID Detection/Model Checkpoint/assets


142/142 [==============================] - 400s 3s/step - loss: 0.4776 - accuracy: 0.7829 - val_loss: 0.3571 - val_accuracy: 0.9086 - lr: 9.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0009000000427477062.
Epoch 2/10
142/142 [==============================] - 398s 3s/step - loss: 0.4509 - accuracy: 0.7931 - val_loss: 0.3253 - val_accuracy: 0.9086 - lr: 9.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0009000000427477062.
Epoch 3/10
142/142 [==============================] - 396s 3s/step - loss: 0.4142 - accuracy: 0.8125 - val_loss: 0.2719 - val_accuracy: 0.8983 - lr: 9.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0008100000384729356.
Epoch 4/10
142/142 [==============================] - ETA: 0s - loss: 0.3765 - accuracy: 0.8414

INFO:tensorflow:Assets written to: /Users/anuragkotiyal/Desktop/Projects/COVID Detection/Model Checkpoint/assets


INFO:tensorflow:Assets written to: /Users/anuragkotiyal/Desktop/Projects/COVID Detection/Model Checkpoint/assets


142/142 [==============================] - 402s 3s/step - loss: 0.3765 - accuracy: 0.8414 - val_loss: 0.2279 - val_accuracy: 0.9348 - lr: 8.1000e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0008100000559352338.
Epoch 5/10
142/142 [==============================] - 402s 3s/step - loss: 0.3677 - accuracy: 0.8452 - val_loss: 0.2233 - val_accuracy: 0.9312 - lr: 8.1000e-04

Epoch 6: LearningRateScheduler setting learning rate to 0.0008100000559352338.
Epoch 6/10
142/142 [==============================] - ETA: 0s - loss: 0.3892 - accuracy: 0.8279

INFO:tensorflow:Assets written to: /Users/anuragkotiyal/Desktop/Projects/COVID Detection/Model Checkpoint/assets


INFO:tensorflow:Assets written to: /Users/anuragkotiyal/Desktop/Projects/COVID Detection/Model Checkpoint/assets


142/142 [==============================] - 398s 3s/step - loss: 0.3892 - accuracy: 0.8279 - val_loss: 0.2035 - val_accuracy: 0.9481 - lr: 8.1000e-04

Epoch 7: LearningRateScheduler setting learning rate to 0.0007290000503417104.
Epoch 7/10
142/142 [==============================] - 409s 3s/step - loss: 0.3573 - accuracy: 0.8390 - val_loss: 0.1988 - val_accuracy: 0.9425 - lr: 7.2900e-04

Epoch 8: LearningRateScheduler setting learning rate to 0.0007290000794455409.
Epoch 8/10
142/142 [==============================] - 398s 3s/step - loss: 0.3491 - accuracy: 0.8488 - val_loss: 0.2141 - val_accuracy: 0.9418 - lr: 7.2900e-04

Epoch 9: LearningRateScheduler setting learning rate to 0.0007290000794455409.
Epoch 9/10
142/142 [==============================] - 435s 3s/step - loss: 0.3337 - accuracy: 0.8485 - val_loss: 0.1859 - val_accuracy: 0.9395 - lr: 7.2900e-04

Epoch 10: LearningRateScheduler setting learning rate to 0.0006561000715009868.
Epoch 10/10
142/142 [=============================

In [16]:
loss, accuracy = vggnet.evaluate(test_data, verbose = 0)

print(f'Final Loss: {loss:.2f} Final Accuracy: {accuracy:.2f}')

Final Loss: 37.14 Final Accuracy: 0.71


In [19]:
def performance_plot(history = None):
  
    legends = ['training', 'validation']
    plt.figure(figsize=(20, 5))

    y1 = history.history['accuracy']
    y2 = history.history['val_accuracy']
  
    plt.subplot(121)
    plt.plot(y1)
    plt.plot(y2)

    plt.title('VGGNET Accuracy\n' + timestamp(1), fontsize=18)
    plt.xlabel('Epochs', fontsize=15)
    plt.ylabel('Accuracy', fontsize=15)
    plt.legend(legends, loc='upper left')
    plt.grid()

    y1 = history.history['loss']
    y2 = history.history['val_loss']

    plt.subplot(122)
    plt.plot(y1)
    plt.plot(y2)

    plt.title('VGGNET Loss\n' + timestamp(1), fontsize=18)
    plt.xlabel('Epochs', fontsize=15)
    plt.ylabel('Loss', fontsize=15)
    plt.legend(legends, loc='upper left')
    plt.grid()

    #plt.savefig('plot.png')

    plt.show()

In [20]:
performance_plot(history = history)

In [82]:
loss, accuracy = vggnet.evaluate(test_data)

print(f'Final Loss: {loss:.2f} Final Accuracy: {accuracy:.2f}')

48/48 [==============================] - 57s 1s/step - loss: 2.8686 - accuracy: 0.6726
Final Loss: 2.87 Final Accuracy: 0.67
